In [1]:
#conda install -c conda-forge gcsfs
#conda install -c conda-forge zarr
# ! pip install xarray
# conda update pandas
# conda install -c conda-forge cftime

In [3]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import zarr
import gcsfs
import seaborn as sns
import os 


In [4]:
xr.set_options(display_style='html')
df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')

df_ta = df.query("activity_id=='ScenarioMIP' & table_id == 'day' & variable_id == 'tas' & experiment_id == 'ssp126'")
print(df_ta)

df_pr = df.query("activity_id=='ScenarioMIP' & table_id == 'day' & variable_id == 'pr' & experiment_id == 'ssp126'")
print(df_pr)

df_tasmax = df.query("activity_id=='ScenarioMIP' & table_id == 'day' & variable_id == 'tasmax' & experiment_id == 'ssp126'")
print(df_tasmax)

df_tasmin = df.query("activity_id=='ScenarioMIP' & table_id == 'day' & variable_id == 'tasmin' & experiment_id == 'ssp126'")
print(df_tasmin)

df_uas = df.query("activity_id=='ScenarioMIP' & table_id == 'day' & variable_id == 'uas' & experiment_id == 'ssp126'")
print(df_uas)

df_vas = df.query("activity_id=='ScenarioMIP' & table_id == 'day' & variable_id == 'vas' & experiment_id == 'ssp126'")
print(df_vas)

df_hurs = df.query("activity_id=='ScenarioMIP' & table_id == 'day' & variable_id == 'hurs' & experiment_id == 'ssp126'")
print(df_hurs)

df_rsds = df.query("activity_id=='ScenarioMIP' & table_id == 'day' & variable_id == 'rsds' & experiment_id == 'ssp126'")
print(df_rsds)

df_sfcWind = df.query("activity_id=='ScenarioMIP' & table_id == 'day' & variable_id == 'sfcWind' & experiment_id == 'ssp126'")
print(df_sfcWind)



        activity_id       institution_id         source_id experiment_id  \
15618   ScenarioMIP            NOAA-GFDL         GFDL-ESM4        ssp126   
54784   ScenarioMIP         CNRM-CERFACS        CNRM-CM6-1        ssp126   
66614   ScenarioMIP                  BCC       BCC-CSM2-MR        ssp126   
68936   ScenarioMIP         CNRM-CERFACS       CNRM-ESM2-1        ssp126   
72721   ScenarioMIP         CNRM-CERFACS        CNRM-CM6-1        ssp126   
...             ...                  ...               ...           ...   
499906  ScenarioMIP  EC-Earth-Consortium  EC-Earth3-Veg-LR        ssp126   
500008  ScenarioMIP  EC-Earth-Consortium  EC-Earth3-Veg-LR        ssp126   
502254  ScenarioMIP                 IPSL   IPSL-CM5A2-INCA        ssp126   
513845  ScenarioMIP                 CMCC         CMCC-ESM2        ssp126   
514457  ScenarioMIP                 NCAR       CESM2-WACCM        ssp126   

       member_id table_id variable_id grid_label  \
15618   r1i1p1f1      day         t

       activity_id institution_id  source_id experiment_id member_id table_id  \
15622  ScenarioMIP      NOAA-GFDL  GFDL-ESM4        ssp126  r1i1p1f1      day   

      variable_id grid_label  \
15622        rsds        gr1   

                                                  zstore  dcpp_init_year  \
15622  gs://cmip6/CMIP6/ScenarioMIP/NOAA-GFDL/GFDL-ES...             NaN   

        version  
15622  20180701  
        activity_id institution_id        source_id experiment_id member_id  \
15621   ScenarioMIP      NOAA-GFDL        GFDL-ESM4        ssp126  r1i1p1f1   
66616   ScenarioMIP            BCC      BCC-CSM2-MR        ssp126  r1i1p1f1   
73111   ScenarioMIP           IPSL     IPSL-CM6A-LR        ssp126  r3i1p1f1   
74830   ScenarioMIP           IPSL     IPSL-CM6A-LR        ssp126  r2i1p1f1   
120179  ScenarioMIP          CCCma          CanESM5        ssp126  r7i1p1f1   
...             ...            ...              ...           ...       ...   
496487  ScenarioMIP           

In [5]:
df_pr['institution_id'].value_counts().to_csv('pr_ssp126_data_available_unique_models.csv')
df_ta['institution_id'].value_counts().to_csv('tas_ssp126_data_available_unique_models.csv')
df_tasmax['institution_id'].value_counts().to_csv('tasmax_ssp126_data_available_unique_models.csv')
df_tasmin['institution_id'].value_counts().to_csv('tasmin_ssp126_data_available_unique_models.csv')
df_uas['institution_id'].value_counts().to_csv('uas_ssp126_data_available_unique_models.csv')
df_vas['institution_id'].value_counts().to_csv('vas_ssp126_data_available_unique_models.csv')
df_hurs['institution_id'].value_counts().to_csv('hurs_ssp126_data_available_unique_models.csv')
df_rsds['institution_id'].value_counts().to_csv('rsds_ssp126_data_available_unique_models.csv')
df_sfcWind['institution_id'].value_counts().to_csv('sfcWind_ssp126_data_available_unique_models.csv')

In [6]:
var_ = ['pr', 'tas', 'tasmax', 'tasmin', 'uas', 'vas', 'hurs', 'rsds', 'sfcWind']
#var_ = ['uas','vas', 'hurs']
for var in var_:
    #print(var, type(var))
    unique_models = pd.read_csv(var+'_ssp126_data_available_unique_models.csv')
    #unique_models.iloc[:,0]
    for i_model,model in enumerate(unique_models.iloc[:,0]):
        #tas_model_ = []
        #print('test')
        #print(type(model))
        #print("table_id == 'day' & variable_id == 'tas' & experiment_id == 'historical' & institution_id =='"+model+"'")
        query_ = "activity_id=='ScenarioMIP' & table_id == 'day' & variable_id == '"+var+"' & experiment_id == 'ssp126' & institution_id =='"+model+"'"
        df_ta = df.query(query_)
        #print('test')
        filename_ = var+'_ssp126_'+str(model)+".nc"
#        filename_ = filename_[:-3]+'_ens_'+str(ens+1)+'.nc'
        #for ens in range(unique_models.iloc[i_model,1]):
        for ens in range(1):
            filename_ = var+'_ssp126_'+str(model)+'_ens_'+str(ens+1)+'.nc'
            count = []
            #print(unique_models.iloc[i_model,1])
            if not os.path.isfile(filename_):
            #for ens in range(1):
            # this only needs to be created once
                
                gcs = gcsfs.GCSFileSystem(token='anon')
                #print('test')
                # get the path to a specific zarr store (the first one from the dataframe above)
                zstore = df_ta.zstore.values[ens] # The index here controls everything, the number

                # create a mutable-mapping-style interface to the store
                mapper = gcs.get_mapper(zstore)

                # open it using xarray and zarr
                ds_ta = xr.open_zarr(mapper, consolidated=True)
                #print(ds_ta.sel(lat=slice(29,32)).sel(lon=slice(261,264)))
                
                try:
                    #tas_hist_ = ds_ta[var].sel(lat=30.3134, lon= 360-97.7666, method='nearest').sel(time=slice('2021','2098'))
                    #print('test 1') #tas_hist_ = ds_ta[var].sel(lat=30.3134, lon= 360-97.7666, method='nearest').sel(time=slice('2021','2098'))
                    tas_hist_ = ds_ta[var].sel(lat=slice(4,14)).sel(lon=slice(2,16)).sel(time=slice('2021','2098'))
                    #print(tas_hist_.values.shape[0]>0) 

                    if tas_hist_.values.shape[0]>0:
                        count.append(ens)
                    #print('After ', tas_hist_.values.shape) 

                except:
                    continue
                
                #print(tas_hist_.time.values.shape[0])
                #print(type(tas_hist_.time.values[0]).__name__)
                #no_of_days=5475
                #print(count)
                
                print("Writing ", filename_)
                out_data = tas_hist_
                out_data[var+'_'] = (('time', 'lat', 'lon'), np.asarray(tas_hist_))
                out_data[var+'_'].to_netcdf(filename_)
#                if tas_hist_.values.shape[0]>0:
#                    if ens==count[0]:
#                        if type(tas_hist_.time.values[0]).__name__=='DatetimeNoLeap':
#                            no_of_days=28470
#                        elif type(tas_hist_.time.values[0]).__name__=='Datetime360Day':
#                            no_of_days=28080
#                        else: 
#                            no_of_days=28489
#                        out_data = tas_hist_
#                else:
#                    continue
#                print('test', no_of_days, tas_hist_.shape)
#                if tas_hist_.values.shape[0]==no_of_days:
#		    # There is a difference of resolution within the same model for some cases
#                    if ens==0 and tas_hist_.shape[1]*tas_hist_.shape[2]<5: 
#                        tas_model_.append(tas_hist_.values)
#			
#                    if ens>0: 
#                        if tas_hist_.shape[1]*tas_hist_.shape[2]<5: 
##                            if tas_model_[0].shape==tas_hist_.shape: 
##                            tas_model_.append(tas_hist_.values) if tas_model_[0].shape==tas_hist_.shape else continue
#                            if tas_model_[0].shape==tas_hist_.shape: tas_model_.append(tas_hist_.values)
#                    if tas_hist_.shape[1]*tas_hist_.shape[2]>4 and not os.path.isfile(filename_[:-3]+'_ens_'+str(ens+1)+'.nc'): 
#                        out_data[var+'_'] = (('time', 'lat', 'lon'), np.asarray(tas_hist_))
#                        out_data[var+'_'].to_netcdf(filename_[:-3]+'_ens_'+str(ens+1)+'.nc')
#                        print('Writing '+filename_[:-3]+'_ens_'+str(ens+1)+'.nc')
#                print('ensemble number: ',ens)
#                ##############################################################################
#            #print('test ', len(tas_model_))
#            if len(tas_model_)>0:
#
##                print(tas_model_[0].shape[0], tas_model_[0].shape[1],tas_model_[0].shape[2])
#                #tas_model__ = np.zeros((tas_model_[0].shape[0], tas_model_[0].shape[1],tas_model_[0].shape[2]))
#                #print(np.nanmean(np.asarray(tas_model_),axis=0).shape)
#                #tas_model__ = np.nanmean(np.asarray(tas_model_),axis=0)
#                #print(tas_model__.shape)
#
#                if tas_hist_.shape[1]*tas_hist_.shape[2]<5:
#                    out_data[var+'_'] = (('time', 'lat', 'lon'), np.nanmean(np.asarray(tas_model_),axis=0))
#                    out_data[var+'_'].to_netcdf(filename_)
#                else:
#                    continue
#                        #out_data[var+'_'] = (('time', 'lat', 'lon'), np.asarray(tas_model_[i_ens]))
#                        #out_data[var+'_'].to_netcdf(filename_[:-3]+'_ens_'+str(i_ens+1)+'.nc')
#                







Writing  pr_ssp126_CCCma_ens_1.nc
Writing  pr_ssp126_MOHC_ens_1.nc
Writing  pr_ssp126_EC-Earth-Consortium_ens_1.nc
Writing  pr_ssp126_MPI-M_ens_1.nc
Writing  pr_ssp126_IPSL_ens_1.nc


C:\ProgramData\Anaconda3\lib\site-packages\xarray\coding\times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
C:\ProgramData\Anaconda3\lib\site-packages\xarray\coding\times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


Writing  pr_ssp126_MRI_ens_1.nc
Writing  pr_ssp126_NCAR_ens_1.nc
Writing  pr_ssp126_CSIRO-ARCCSS_ens_1.nc
Writing  pr_ssp126_CNRM-CERFACS_ens_1.nc
Writing  pr_ssp126_NIMS-KMA_ens_1.nc
Writing  pr_ssp126_NUIST_ens_1.nc
Writing  pr_ssp126_CMCC_ens_1.nc
Writing  pr_ssp126_INM_ens_1.nc
Writing  pr_ssp126_NCC_ens_1.nc
Writing  pr_ssp126_KIOST_ens_1.nc
Writing  pr_ssp126_NOAA-GFDL_ens_1.nc
Writing  pr_ssp126_CAMS_ens_1.nc
Writing  pr_ssp126_DWD_ens_1.nc
Writing  pr_ssp126_DKRZ_ens_1.nc
Writing  pr_ssp126_AWI_ens_1.nc
Writing  pr_ssp126_BCC_ens_1.nc
Writing  pr_ssp126_CCCR-IITM_ens_1.nc
Writing  tas_ssp126_MIROC_ens_1.nc
Writing  tas_ssp126_CCCma_ens_1.nc
Writing  tas_ssp126_MOHC_ens_1.nc
Writing  tas_ssp126_EC-Earth-Consortium_ens_1.nc
Writing  tas_ssp126_CNRM-CERFACS_ens_1.nc
Writing  tas_ssp126_MPI-M_ens_1.nc
Writing  tas_ssp126_IPSL_ens_1.nc
Writing  tas_ssp126_CAS_ens_1.nc
Writing  tas_ssp126_NCAR_ens_1.nc
Writing  tas_ssp126_CSIRO-ARCCSS_ens_1.nc
Writing  tas_ssp126_NIMS-KMA_ens_1.nc
Wr

C:\ProgramData\Anaconda3\lib\site-packages\xarray\coding\times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
C:\ProgramData\Anaconda3\lib\site-packages\xarray\coding\times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


Writing  tas_ssp126_MRI_ens_1.nc
Writing  tas_ssp126_CAMS_ens_1.nc
Writing  tas_ssp126_DWD_ens_1.nc
Writing  tas_ssp126_DKRZ_ens_1.nc
Writing  tas_ssp126_AWI_ens_1.nc
Writing  tas_ssp126_BCC_ens_1.nc
Writing  tas_ssp126_CCCR-IITM_ens_1.nc
Writing  tasmax_ssp126_MIROC_ens_1.nc
Writing  tasmax_ssp126_CCCma_ens_1.nc
Writing  tasmax_ssp126_EC-Earth-Consortium_ens_1.nc
Writing  tasmax_ssp126_MPI-M_ens_1.nc
Writing  tasmax_ssp126_IPSL_ens_1.nc


...

Writing  vas_ssp126_MRI_ens_1.nc
Writing  vas_ssp126_KIOST_ens_1.nc
Writing  vas_ssp126_DKRZ_ens_1.nc
Writing  vas_ssp126_CSIRO-ARCCSS_ens_1.nc
Writing  vas_ssp126_AWI_ens_1.nc
Writing  vas_ssp126_BCC_ens_1.nc
Writing  vas_ssp126_CCCR-IITM_ens_1.nc
Writing  hurs_ssp126_CCCma_ens_1.nc
Writing  hurs_ssp126_MOHC_ens_1.nc
Writing  hurs_ssp126_EC-Earth-Consortium_ens_1.nc
Writing  hurs_ssp126_CNRM-CERFACS_ens_1.nc
Writing  hurs_ssp126_MPI-M_ens_1.nc
Writing  hurs_ssp126_IPSL_ens_1.nc
Writing  hurs_ssp126_NCAR_ens_1.nc
Writing  hurs_ssp126_CSIRO-ARCCSS_ens_1.nc
Writing  hurs_ssp126_NIMS-KMA_ens_1.nc
Writing  hurs_ssp126_CMCC_ens_1.nc
Writing  hurs_ssp126_INM_ens_1.nc
Writing  hurs_ssp126_NCC_ens_1.nc
Writing  hurs_ssp126_NOAA-GFDL_ens_1.nc


C:\ProgramData\Anaconda3\lib\site-packages\xarray\coding\times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
C:\ProgramData\Anaconda3\lib\site-packages\xarray\coding\times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


Writing  hurs_ssp126_MRI_ens_1.nc
Writing  hurs_ssp126_DWD_ens_1.nc
Writing  hurs_ssp126_DKRZ_ens_1.nc
Writing  hurs_ssp126_AWI_ens_1.nc
Writing  hurs_ssp126_CCCR-IITM_ens_1.nc
Writing  rsds_ssp126_NOAA-GFDL_ens_1.nc
Writing  sfcWind_ssp126_CCCma_ens_1.nc
Writing  sfcWind_ssp126_MOHC_ens_1.nc
Writing  sfcWind_ssp126_MPI-M_ens_1.nc
Writing  sfcWind_ssp126_IPSL_ens_1.nc
Writing  sfcWind_ssp126_MIROC_ens_1.nc
Writing  sfcWind_ssp126_NCAR_ens_1.nc
Writing  sfcWind_ssp126_NIMS-KMA_ens_1.nc
Writing  sfcWind_ssp126_CSIRO-ARCCSS_ens_1.nc
Writing  sfcWind_ssp126_CMCC_ens_1.nc
Writing  sfcWind_ssp126_INM_ens_1.nc
Writing  sfcWind_ssp126_EC-Earth-Consortium_ens_1.nc
Writing  sfcWind_ssp126_NOAA-GFDL_ens_1.nc
Writing  sfcWind_ssp126_NCC_ens_1.nc
Writing  sfcWind_ssp126_DWD_ens_1.nc


C:\ProgramData\Anaconda3\lib\site-packages\xarray\coding\times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
C:\ProgramData\Anaconda3\lib\site-packages\xarray\coding\times.py:673: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


Writing  sfcWind_ssp126_MRI_ens_1.nc
Writing  sfcWind_ssp126_BCC_ens_1.nc
Writing  sfcWind_ssp126_DKRZ_ens_1.nc
Writing  sfcWind_ssp126_AWI_ens_1.nc
Writing  sfcWind_ssp126_CCCR-IITM_ens_1.nc
